# GACSE Workshop

In [91]:
from __future__ import print_function

In [92]:
import sys
sys.path.append('../build/')

In [96]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [97]:
import versor as vsr

In [98]:
from motor_estimation import MotorEstimationSolver

## Ground truth motor

In [99]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


## Motor Estimation from Points

### Dataset generation

In [100]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

### Estimate motor

In [101]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(points_a, points_b):
    mes.add_point_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary) = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 5, Initial cost: 2.177005e+01, Final cost: 2.869924e-17, Termination: CONVERGENCE


#### Check solution

In [9]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))

[ 0.866  0.    -0.5    0.    -0.683 -0.433 -0.183 -0.25 ]
[ 0.866 -0.    -0.5   -0.    -0.683 -0.433 -0.183 -0.25 ]
True


## Motor Estimation from Lines

### Dataset generation

In [102]:
lines_a = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)))
          for i in range(10)]
lines_b = [line.spin(motor) for line in lines_a]

### Estimate motor using 6 residuals

$$ \arg \min_M \frac{1}{2} \sum_{i=1}^n \left| M \varLambda_{b_i} \tilde{M} - \varLambda_{a_i}  \right| $$

In [103]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 6, Initial cost: 4.007902e+01, Final cost: 6.447643e-20, Termination: CONVERGENCE


#### Check solution

In [104]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))

[ 0.866  0.    -0.5    0.    -0.683 -0.433 -0.183 -0.25 ]
[ 0.866 -0.    -0.5    0.    -0.683 -0.433 -0.183 -0.25 ]
True


### Estimate motor using 4 residuals

In [13]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 20, Initial cost: 4.291739e+01, Final cost: 1.250000e+00, Termination: CONVERGENCE


#### Check solution

In [14]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))
print(np.allclose(np.array(motor), np.array(estimated_motor), 
                  rtol=1e-05, atol=1e-03))

[ 0.866  0.    -0.5    0.    -0.683 -0.433 -0.183 -0.25 ]
[ 0.866   0.0002 -0.5001  0.0002 -0.6832 -0.4329 -0.1828 -0.2502]
False
True


### Estimate motor using 2 residuals

In [15]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_norm_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 15, Initial cost: 4.291739e+01, Final cost: 1.250002e+00, Termination: CONVERGENCE


#### Check solution

In [16]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))
print(np.allclose(np.array(motor), np.array(estimated_motor), 
                  rtol=1e-05, atol=1e-03))

[ 0.866  0.    -0.5    0.    -0.683 -0.433 -0.183 -0.25 ]
[ 0.8658  0.0001 -0.5003  0.0001 -0.683  -0.4329 -0.1827 -0.2503]
False
True


## Motor Estimation from planes

### Dataset generation

In [110]:
n_planes = 10
planes_a = []
for i in range(10):
    dir_vec = np.random.normal(0.0, 0.8, 4)
    dir_vec = dir_vec / np.linalg.norm(dir_vec)
    distance = np.random.normal(0.0, 0.8,1)
    arr = dir_vec + distance
    planes_a.append(vsr.Dlp(dir_vec[0],dir_vec[1],dir_vec[2],distance[0]))
planes_b = [plane.spin(motor) for plane in planes_a]

### Estimate motor using 2 residuals

In [111]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
noise_motor_plane = vsr.Vec(0.1,0.1,0.1).trs() * vsr.Rot(vsr.Biv(1,0,0) * np.pi/24)
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(planes_a, planes_b):
#     mes.add_dual_plane_angle_error_residual_block(a,b.spin(noise_motor_plane))
    mes.add_dual_plane_angle_error_residual_block(a,b)

mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.function_tolerance = 1e-8
mes.max_num_iterations = 100
em, summary = mes.solve()
print(motor)
print(em)
print(summary['full_report'])

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]
Mot: [ 0.87 7.3e-06 -0.5 -1.1e-05 -0.68 -0.43 -0.18 -0.25 ]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                   20                       20

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver thread

## Motor Estimation from tangents

### Dataset generation

In [19]:
n_tangents = 10
tangents_a = []
for i in range(10):
    dir_vec = np.random.normal(0.0, 0.8, 3)
    dir_vec = dir_vec / np.linalg.norm(dir_vec)
    pos = np.random.normal(0.0, 0.8, 3)
    tangents_a.append(vsr.Tnv(vsr.Vec(*dir_vec)).spin(vsr.Vec(*pos).trs() * 
                                             vsr.Rot(1,0,0,0)))

tangents_b = [tangent.spin(motor) for tangent in tangents_a]

### Estimate motor

In [20]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(tangents_a, tangents_b):
    mes.add_tangent_vector_point_angle_error_residual_block(a,b)

mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
# mes.function_tolerance = 1e-8
mes.max_num_iterations = 200
estimated_motor, summary = mes.solve()
print(initial_motor)
print(motor)
print(estimated_motor)
print(summary['full_report'])

Mot: [ 1 0 0 0 0 0 0 0 ]
Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]
Mot: [ 0.87 8.9e-09 -0.5 3.4e-08 -0.68 -0.43 -0.18 -0.25 ]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                   60                       60

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                      

## Motor Estimation from heterogeneous conformal objects

In [21]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
noise_motor_plane = vsr.Vec(0.05,0.05,0.05).trs() * vsr.Rot(vsr.Biv(1,0,0) * np.pi/100)
noise_motor_lines = vsr.Vec(-0.01,-0.01,-0.01).trs() * vsr.Rot(vsr.Biv(0,1,0) * (-np.pi/100))
noise_motor_points = vsr.Vec(0.1,0.1,0.1).trs() * vsr.Rot(vsr.Biv(0,0,1) * (np.pi/100))
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(planes_a, planes_b):
    mes.add_dual_plane_angle_error_residual_block(a,b.spin(noise_motor_plane))
for a, b in zip(lines_a, lines_b):
#     mes.add_line_angle_distance_residual_block(a, b.spin(noise_motor_lines))
    mes.add_line_correspondences_residual_block(a,b.spin(noise_motor_lines))
for a, b in zip(points_a, points_b):
    mes.add_point_correspondences_residual_block(a,b.spin(noise_motor_points))
for a, b in zip(tangents_a, tangents_b):
    mes.add_tangent_vector_point_angle_error_residual_block(a,b)


# for a, b in zip(points_a, points_b):
#     mes.add_point_correspondences_residual_block(a,b)
    
    

mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.function_tolerance = 1e-8
mes.max_num_iterations = 100
em, summary = mes.solve()
print(initial_motor)
print(motor)
print(em)
print(summary['full_report'])

Mot: [ 1 0 0 0 0 0 0 0 ]
Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]
Mot: [ 0.87 -0.0095 -0.49 -0.013 -0.71 -0.44 -0.18 -0.24 ]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            40                       40
Residual                                  170                      170

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                      

## Hand Eye Calibration

### Set up motors

In [22]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(0.1, 0.2, 0.3).trs() * vsr.Biv(0.0,1.0,0.0).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

### Dataset generation

In [23]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
            LAs.append(((As[j].rev() * As[i]).log() * 0.5).unit())
            LBs.append(((Bs[j] * Bs[i].rev()).log() * 0.5).unit())
    return LAs, LBs

In [24]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)
print(len(lines_a))

45


### Estimate hand-eye motor 

In [25]:
mes = MotorEstimationSolver(M_eye_in_hand_initial)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.function_tolerance = 1e-16
estimated_motor, summary = mes.solve()
print(summary['full_report'])


Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            45                       45
Residual                                  180                      180

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1

Cost:
Initial                          7.469222e+01
Fin

#### Check solution

In [26]:
print(np.array(M_eye_in_hand))
print(np.array(estimated_motor))
print(np.allclose(np.array(M_eye_in_hand), np.array(estimated_motor)))

[ 0.5403  0.     -0.8415  0.     -0.1532 -0.054  -0.039  -0.0841]
[ 0.5403 -0.      0.8415 -0.      0.1532  0.054   0.039  -0.0841]
False


### Real Experiment

In [27]:
import cPickle as pickle
import math3d as m3d

#### Dataset generation

In [28]:
rob_pose_list = pickle.load(open('pose_list.dump'))
obj_pose_list = pickle.load(open('obj_pose_list.dump'))
a_trfs = pickle.load(open('as.dump'))
b_trfs = pickle.load(open('bs.dump'))

In [29]:
rob_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in rob_pose_list]]
obj_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in obj_pose_list]]

In [30]:
n_lines = len(rob_motors)
lines_a = []
lines_b = []
for i in range(n_lines):
    for j in range(i+1,n_lines):
        lines_a.append(((rob_motors[j].rev() * rob_motors[i]).log() * 0.5).unit())
        lines_b.append(((obj_motors[j].rev() * obj_motors[i]).log() * 0.5).unit())

In [31]:
n_lines = len(a_trfs)
motors_a = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in a_trfs]]
motors_b = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in b_trfs]]
lines_a = [motor.log() for motor in motors_a]
lines_b = [motor.log() for motor in motors_b]

#### Estimate hand-eye motor

In [32]:
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(b,a)
#     mes.add_line_angle_distance_residual_block(b,a)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.max_num_iterations = 25
mes.function_tolerance = 1e-12
estimated_motor, summary = mes.solve()
print(summary['full_report'])


Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            20                       20
Residual                                  120                      120

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1

Cost:
Initial                          3.395318e+00
Fin

In [45]:
print(estimated_motor)
print(m3d.Transform(estimated_motor.matrix()))

Mot: [ 0.0028 -0.0027 -0.68 -0.74 -0.053 -0.058 0.05 -0.018 ]
<Transform:
<Orientation: 
array([[ 0.0877, -0.9961,  0.0002],
       [-0.9961, -0.0877, -0.0079],
       [ 0.0079,  0.0005, -1.    ]])>
<Vector: (0.04172, 0.09897, 0.15780)>
>


#### Combination of estimates

In [49]:
bivectors = [np.array((r * estimated_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
m = vsr.Dll(*np.average(bivectors,0)).exp()
print(m3d.Transform(m.matrix()))

<Transform:
<Orientation: 
array([[ 0.7348,  0.0085,  0.6783],
       [-0.6783,  0.0001,  0.7348],
       [ 0.0062, -1.    ,  0.0058]])>
<Vector: (-0.98505, -0.87728, 0.75166)>
>


#### Tsai Lenz Solver

In [47]:
from tsai_lens_calibration import TsaiLenzCalibrator

In [73]:
pose_pairs = [(m3d.Transform(A), m3d.Transform(B)) 
              for (A, B)  in zip(rob_pose_list, obj_pose_list)]
tsai_lens = TsaiLenzCalibrator(pose_pairs)
tsai_lens.sensor_in_flange

<Transform:
<Orientation: 
array([[ 0.1246, -0.992 , -0.0178],
       [-0.9841, -0.1213, -0.1294],
       [ 0.1263,  0.0336, -0.9914]])>
<Vector: (0.02149, 0.07718, 0.14263)>
>

#### Park Martin Solver

In [83]:
from park_martin import calibrate as park_martin_calibration
A, B = [], []
n_trfs = len(rob_pose_list)
for i in range(n_trfs):
    for j in range(i+1, n_trfs):
        A.append((m3d.Transform(rob_pose_list[j]).inverse * m3d.Transform(rob_pose_list[i])).array)
        B.append((m3d.Transform(obj_pose_list[j]).inverse * m3d.Transform(obj_pose_list[i])).array)
R,t = park_martin_calibration(A,B)
print(m3d.Transform(R,t))

<Transform:
<Orientation: 
array([[ 0.1051, -0.9945,  0.0021],
       [-0.9943, -0.1051, -0.0175],
       [ 0.0176, -0.0003, -0.9998]])>
<Vector: (0.04123, 0.09907, 0.14488)>
>


In [112]:
def daniilidis(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        LA = LA.unit()
        LB = LB.unit()
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]
        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    
    [U, s, Vt] = np.linalg.svd(Ds)
    v7 = Vt.T[:,-2]
    v8 = Vt.T[:,-1]
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * v7 + lambda2 * v8
    m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
    m_arr[4:8] = np.roll(m_arr[4:8], -1)
    
    print(Ds)
    
    return vsr.Mot(*m_arr)

In [113]:
daniilidis_motor = daniilidis(lines_b, lines_a)

[[ 0.6974  0.      1.208  -0.3048  0.      0.      0.      0.    ]
 [ 0.     -1.208   0.      1.2129  0.      0.      0.      0.    ]
 [-0.7003  0.3048 -1.2129  0.      0.      0.      0.      0.    ]
 [-1.7605  0.     -1.7661  0.267   0.6974  0.      1.208  -0.3048]
 [ 0.6964  1.7661  0.      2.3747  0.     -1.208   0.      1.2129]
 [-0.8029 -0.267  -2.3747  0.     -0.7003  0.3048 -1.2129  0.    ]
 [-0.5363  0.     -0.929   1.2755  0.      0.      0.      0.    ]
 [ 0.      0.929   0.      0.9575  0.      0.      0.      0.    ]
 [-0.5528 -1.2755 -0.9575  0.      0.      0.      0.      0.    ]
 [-0.7863  0.     -1.9677 -1.2322 -0.5363  0.     -0.929   1.2755]
 [-0.935   1.9677  0.      0.181   0.      0.929   0.      0.9575]
 [ 1.5398  1.2322 -0.181   0.     -0.5528 -1.2755 -0.9575  0.    ]
 [-0.1995  0.     -0.3455 -1.8307  0.      0.      0.      0.    ]
 [-0.      0.3455  0.     -0.6058  0.      0.      0.      0.    ]
 [ 0.3498  1.8307  0.6058  0.      0.      0.      0.      0. 

In [84]:
print(m3d.Transform(daniilidis_motor.matrix()))

<Transform:
<Orientation: 
array([[ 0.1293, -0.9915, -0.0128],
       [-0.9912, -0.1289, -0.0297],
       [ 0.0278,  0.0166, -0.9995]])>
<Vector: (0.11194, 0.08841, 0.13725)>
>


In [86]:
print(estimated_motor * estimated_motor.rev())
print(daniilidis_motor * daniilidis_motor.rev())

Mot: [ 1 0 0 0 0 0 -7.6e-18 2.6e-17 ]
Mot: [ 1 0 0 0 -8.7e-18 -1.7e-18 4.8e-18 0.00081 ]


In [87]:
bivectors = [np.array((r * daniilidis_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
m = vsr.Dll(*np.average(bivectors,0)).exp()
print(m3d.Transform(m.matrix()))

<Transform:
<Orientation: 
array([[ 0.7232,  0.0245,  0.6902],
       [-0.6891, -0.041 ,  0.7235],
       [ 0.0461, -0.9989, -0.0128]])>
<Vector: (-0.97112, -0.86143, 0.83679)>
>
